In [1]:
#import gensim.function

text=open('Стихи.txt','r').readlines()

In [2]:
text = [i.strip() for i in text]

In [3]:
text = [i for i in text if i != '' and 'Стихотворения' not in i]
text = text[:len(text)-243]
data=''.join(text)

In [4]:
len(data)

603368

In [5]:
data = data.replace('\xa0',' ')

for number in '123456789':
    data=data.replace(number,'0')
for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    data=data.replace(english_letter, '')
for english_letter in 'abcdefghijklmnopqrstuvwzyz':
    data=data.replace(english_letter, '')    

In [6]:
b=set(data)
c=dict()
for symbol in b:
    c[symbol] = data.count(symbol)
import operator
c1 = sorted(c.items(), key=operator.itemgetter(1))

In [7]:
Alphabet =''.join(c.keys())
Alphabet
#We don't delete any of the chars as now

'ёвРтx]ъ)ЦЕ_МЛДСЙ!ч0сЯ#Х/юЗТЖ,Пе\'ьЭЩл«НШ-д"КзмгА;ЮУ(о„–рБ*:пя .Чэжщ»>ОаыкхВн&Гц?шйу<Фб—[И…иф'

In [3]:

def string_vectorizer(string, alphabet=Alphabet):
    vector = [[0 if char != letter else 1 for char in Alphabet] 
                  for letter in string]
    return vector
def vector_stringizer(vector, alphabet = Alphabet,return_first=True):
    answ=''
    if return_first:
        index = vector.argsort()[-1]
        answ+=Alphabet[index]
    else:
        index = vector.argsort()[-2]
        answ+=Alphabet[index]
    return answ

In [9]:

data_onehot = string_vectorizer(data)

In [2]:
import keras
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Flatten,Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from tqdm import tqdm
from scipy import sparse

C:\Users\DK\Anaconda3.1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
window_len = 15
X_total,Y_total=[],[]
i=1000
X_total.append(data_onehot[i-window_len:i])
Y_total.append(data_onehot[i])

In [12]:
def make_dataset(data_onehot):
    X_total,Y_total=[],[]
    for i in range(window_len, len(data_onehot)):
        X_total.append(data_onehot[i-window_len:i])
        Y_total.append(data_onehot[i])
    return np.array(X_total), np.array(Y_total)
X_total, Y_total=make_dataset(data_onehot)

In [13]:
X_total.shape

(595977, 15, 91)

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X_total, Y_total, test_size=0.15, random_state = 1)
del X_total
del  Y_total

In [7]:
from  keras.callbacks import EarlyStopping
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model.add(Flatten())
model.add(Dense(len(Alphabet),activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()
model.fit(X_train,Y_train, epochs=20,validation_data = (X_test, Y_test), batch_size=10, verbose=1,callbacks=[callback])

In [ ]:
model.save('Pushkin1.h5')  # creates a HDF5 file 'my_model.h5'
  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('Pushkin1.h5')

In [93]:
text = 'Уж небо осенью '
from copy import deepcopy
text1 = deepcopy(text)

Предсказываем наиболее вероятную букву

In [96]:
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    predicted_char = vector_stringizer(predicted_vector[0])
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text)    

Уж небо осенью тобой.И вдруг и старик последний свободы не стала в силах восторгом забытый под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,

In [99]:
text = 'Уж небо осенью '
text1 = deepcopy(text)

Предсказываем вторую по вероятности букву

In [100]:
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью свенцы, вечно тем прельем, прездучивых,Изманы,Помнешнному,Стоя поэзь тревеще небо в тем нахмерелых, скорее прелосая, всё,Всемурощей,Но,,Неталямь своей,Истиг в послуждалскою,Несестителях,Их,[час на стигах,Смолотен мирногде, никуплата нежиною твеяжь в придаю,Измалася,Дрогарайска прияжно мною послужны души,Или,Чадал из ни слог незносадей,Невинов днему.Явлеющей,Или,,С свердницами,С какновики,Возвождатей,Невиненно слогой,Или,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

Предсказываем наиболее вероятную букву, но не всегда.

In [102]:
text = 'Уж небо осенью '
text1 = deepcopy(text)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    if i%15!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью столь, и тому подъемлет он в сень на постали мой друг, и славы постали в прах веселых под шумит,И все дару под сень на свободы не стала,И сладостный простой пред тобы томный старик последний свобы в силой друг,В том за староских последний стал и сладость в полязу под себя возвышенный свободы,На моренья милый друг,Придать в свои под сень мой драгой старик постали в приходит,Он в своей прелестной полной старик последним простой мой долго страстной старик под сень на стал и старого столеты,В мечтанье старик и столковал он в пред ним столем постали в прах веселых под шумит,И все дару под сень на свободы не стала,И сладостный простой пред тобы томный старик последний свобы в силой друг,В том за староских последний стал и сладость в полязу под себя возвышенный свободы,На моренья милый друг,Придать в свои под сень мой драгой старик постали в приходит,Он в своей прелестной полной старик последним простой мой долго страстной старик под сень на стал и старого столеты,В мечтанье ст

Все равно циклится. А если выбирать вторую букву вместо первой не всегда,а более-менее рандомно?

In [106]:
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(0)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    j = random.randint(0,30)
    if j%25!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью тоба восторгом полетит, и странник мой,Как был под слезы просто полной старик послушный,Над собности подавал в приятной другием восторгом стоит простой мой друг, и сладостный, не страдал восторгом столом полетит,И в сердца и старик,Но ты, как девица страдала.Стал и сладостной постали в пред колени полный светил восторгом столетом заботы,На стану в молчанье, восторгом столом последний расстал и полнощный сводом.И странный, воскресный поэт,Но с подруги полной старик последний свободы не стали в полезный под сени, столицы стол,На полны на странного страдал, от бранный старик последний свои постали, не стройный красной?Ужель ужа ли в столе приведет.Посталься в другим столиться, в столе мой придет от приведеньем последний свободы не стала проститься на кругом полетит,И старик мой проститель полетели,И все не страда вознастить, постали в приятным столицы молитвый,В получил с невозок на странник старик, под скалоть, мой друг, издаливый свобы в сили в постеле под сень мира,И стр

Тоже бред, но по крайней мере не циклится. Уже лучше.

Дубль два. Учим более сложную модель, но с дропаутом. 

In [24]:
window_len = 15
model1 = Sequential()
model1.add(LSTM(120, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model1.add(Flatten())
model1.add(Dropout(0.1))
model1.add(Dense(len(Alphabet),activation='softmax'))
model1.compile(loss='binary_crossentropy', optimizer='adam')
model1.summary()
model1.fit(X_train,Y_train, epochs=20,validation_data = (X_test, Y_test), batch_size=10, verbose=1,callbacks=[callback])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 15, 120)           101760    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1800)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 91)                163891    
Total params: 265,651
Trainable params: 265,651
Non-trainable params: 0
_________________________________________________________________
Train on 506580 samples, validate on 89397 samples
Epoch 1/20
506580/506580 [==============================] - 530s 1ms/step - loss: 0.0371 - val_loss: 0.0342
Epoch 2/20
506580/506580 [==============================] - 547s 1ms/step - loss: 0.0334 - val_loss: 0.0325
E

KeyboardInterrupt: 

Проверяем.

In [25]:
model1.save('Pushkin2.h5')

In [28]:
from copy import deepcopy
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(0)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model1.predict(text_array)
    j = random.randint(0,40)
    if j%35!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью слава,И всё по светлом полетал,И с под сенью, поступный возведь весь веселый взосленных,С первых лет под волновал своей положит,И с тобой пред ним странный свет под вечерный полон,В своих под сенью подобной полон воздохнет страна своему странный свет под вечерный полон вознеслись под себя возврата не смеря,Не прихот и в подарился.Но всё тебя не вознеслись под сенью:Но скоро ль на сверенный светел,Иль в сердце послушная под старик, в тебя в последний рад, с тобою, странной приходит в полуночи,И сладострастья в сердце пред ним своем под сенью подобной полон воздохнуть под сенью на свое подарит:Без под вечерней под сени,Свои пред тобою,И слезы с подобрать в последний разговор,И сладострастной струится в темной под своих день,И в сердце простой верный свет, —Не вознес и волны.Пред ним старый свое восторгов прихоти не приходит на берег восторгов прихоти не приходит на свое вечерней,И сладостно странный страстному странный старик,Под сенью воспоминание странный свой тебя в пос

Вроде, немного получше. А что если расширить выборку? Добавить к стихам поэмы?

In [5]:
from tqdm import tqdm
text=open('Стихи+Поэмы.txt','r').readlines()
text = [i for i in text if i != '' and 'Стихотворения' not in i]
text = [i.strip() for i in text]
text = text[:len(text)-243]
data=''.join(text)
data = data.replace('\xa0',' ')

for number in '123456789':
    data=data.replace(number,'0')
for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    data=data.replace(english_letter, '')
for english_letter in 'abcdefghijklmnopqrstuvwzyz':
    data=data.replace(english_letter, '')    
b=set(data)
c=dict()
for symbol in b:
    c[symbol] = data.count(symbol)
import operator
c1 = sorted(c.items(), key=operator.itemgetter(1))
Alphabet =''.join(c.keys())
Alphabet
#We don't delete any of the chars as now

data_onehot = string_vectorizer(data)
window_len = 15
X_total,Y_total=[],[]
def make_dataset(data_onehot):
    X_total,Y_total=[],[]
    for i in tqdm(range(window_len, len(data_onehot))):
        X_total.append(data_onehot[i-window_len:i])
        Y_total.append(data_onehot[i])
    return np.array(X_total), np.array(Y_total)
X_total, Y_total=make_dataset(data_onehot)
del tqdm

100%|████████████████████████████| 1066406/1066406 [00:04<00:00, 220970.99it/s]


In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X_total, Y_total, test_size=0.15, random_state = 1)
del X_total
del  Y_total

In [ ]:
import sys
del sys.modules["tqdm"]

In [17]:
tqdm

NameError: name 'tqdm' is not defined

In [19]:
window_len = 15
model2 = Sequential()
model2.add(LSTM(120, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model2.add(Flatten())
model2.add(Dropout(0.1))
model2.add(Dense(len(Alphabet),activation='softmax'))
model2.compile(loss='binary_crossentropy', optimizer='adam')
model2.summary()
model2.fit(X_train,Y_train, epochs=20,validation_data = (X_test, Y_test), batch_size=10, verbose=2,callbacks=[callback])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 15, 120)           103680    
_________________________________________________________________
flatten_6 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1800)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 95)                171095    
Total params: 274,775
Trainable params: 274,775
Non-trainable params: 0
_________________________________________________________________
Train on 906445 samples, validate on 159961 samples
Epoch 1/20
 - 891s - loss: 0.0342 - val_loss: 0.0314
Epoch 2/20
 - 923s - loss: 0.0309 - val_loss: 0.0298
Epoch 3/20
 - 855s - loss: 0.0297 - val_loss: 0.0293
Epoch 4/20
 - 844s - loss: 0.0291 - val_loss: 0.0289
Epoc

In [27]:
from copy import deepcopy
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(0)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,len(Alphabet))
    predicted_vector = model2.predict(text_array)
    j = random.randint(0,40)
    if j%35!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью возводит, не стал под сень меньшой полетела с ней.Он расстали в серебранилась,Катарая сердцу полетели волны,И в своей простились в поле в поле в поле в поле в поле в поле в поле в стране славы старик на столом,И страх и в поле воспомической странный славы,Не старый восторгов своей на своей долго в темной под сень моим образ моих страны славой, друзья, приятной страх на свете не смело.Он в старик с после в сердце полный взор,Покрытый старины,Как был он не больше был он не стал под сень моей друг, не стал пред ним страны славы, страх на старуха своей простой странный славы,Не старость он не смелой странный славы,Не стоят на старик на берег восточный простой не смелой странный славы,Не старый восторгов своей на свет под сень моей друг, не стал под сени моей друг, славы бессмертный прихладный, друзья, приветство в страх не слезы в том не смелой странный славы,Не старый воспоминанье,И страшит сердце волны светлой страны славы,Не стал под сень моей душой волной,И полный взор е

In [23]:
len(text1)

15

In [25]:
string_vectorizer(text1))

15